In [171]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.linalg import solve
from typing import Dict, List, Tuple

from copy import deepcopy

import random
from math import floor
from sklearn.preprocessing import normalize
from sklearn.utils.extmath import randomized_svd
from rich import print

get_ipython().ast_node_interactivity = "all"

In [172]:
from typing import TypedDict

from pandas import DataFrame

PLIERResults = TypedDict(
    "PlierResults",
    {
        "residual": DataFrame,
        "B": DataFrame,
        "Z": DataFrame,
        "U": DataFrame,
        "C": DataFrame,
        "L1": float,
        "L2": float,
        "L3": float,
        "heldOutGenes": Dict[str, List[str]],
        "withPrior": Dict[str, int],
        "Uauc": DataFrame,
        "Up": DataFrame,
        "summary": DataFrame,
    },
)

In [173]:
data_dir = Path("/Users/milessmith/workspace/pyplier/tests/data/plier/")

In [174]:
allPaths = pd.read_csv(data_dir / "allPaths.csv", index_col=0, header=0)
dataWholeBlood = pd.read_csv(data_dir / "dataWholeBlood.csv", index_col=0, header=0)

In [175]:
allPaths.head()

,IRIS_Bcell-Memory_IgG_IgA,IRIS_Bcell-Memory_IgM,IRIS_Bcell-naive,IRIS_CD4Tcell-N0,IRIS_CD4Tcell-Th1-restimulated12hour,IRIS_CD4Tcell-Th1-restimulated48hour,IRIS_CD4Tcell-Th2-restimulated12hour,IRIS_CD4Tcell-Th2-restimulated48hour,IRIS_CD8Tcell-N0,IRIS_DendriticCell-Control,...,KEGG_GNRH_SIGNALING_PATHWAY,KEGG_BASAL_TRANSCRIPTION_FACTORS,REACTOME_SYNTHESIS_OF_DNA,KEGG_HEMATOPOIETIC_CELL_LINEAGE,KEGG_T_CELL_RECEPTOR_SIGNALING_PATHWAY,PID_IL4_2PATHWAY,REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,PID_BCR_5PATHWAY,PID_TELOMERASEPATHWAY,PID_PI3KPLCTRKPATHWAY
gene,,,,,,,,,,,,,,,,,,,,,
GAS6,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
MMP14,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
MARCKSL1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SPARC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CTSD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [176]:
dataWholeBlood.head()

,BD8001,BD8002,BD8003,BD8004,BD8005,BD8006,BD8007,BD8008,BD8009,BD8010,...,BD8031,BD8032,BD8033,BD8034,BD8038,BD8041,BD8042,BD8043,BD8044,BD8045
gene,,,,,,,,,,,,,,,,,,,,,
GAS6,7.123563,7.846633,8.356313,7.387916,7.859675,7.057541,8.960098,8.120199,7.061915,7.467680,...,7.837566,7.576831,7.589572,7.751689,7.764586,7.526250,7.144113,7.874906,7.362536,8.315021
MMP14,6.636157,7.523565,7.033673,6.895476,6.860524,7.268107,7.121380,7.196859,6.764261,7.788809,...,7.026309,6.822608,7.197420,7.060375,6.712708,6.758632,6.506008,7.371407,7.299790,7.354577
MARCKSL1,10.632837,11.208832,10.519870,10.804867,10.940891,10.984602,11.258157,10.943532,11.242130,11.001261,...,10.540350,11.192079,10.913452,10.910316,10.731734,10.960063,10.852984,10.863946,11.093211,11.404975
SPARC,12.206811,11.462327,12.391210,12.457026,12.036049,12.010138,11.342475,10.979392,12.464739,11.297043,...,10.445423,11.940918,12.454682,11.457722,11.688371,11.563955,12.331355,11.409792,11.690587,12.448106
CTSD,13.147963,13.218464,12.574546,12.710222,13.151780,13.131948,13.466095,13.326034,12.885950,13.818859,...,12.837019,13.388076,12.851351,12.840063,12.448499,13.271724,12.993058,13.506943,13.312904,14.070121


In [177]:
dataPath = Path("/Users/milessmith/workspace/pyplier/tests/data/common")
data = pd.read_csv(filepath_or_buffer=dataPath / "plierRes_data.csv", index_col=0)

In [178]:
plierRes_b = pd.read_csv(filepath_or_buffer=dataPath / "plierRes_b.csv", index_col=0)
plierRes_c = pd.read_csv(filepath_or_buffer=dataPath / "plierRes_c.csv", index_col=0)
plierRes_residual = pd.read_csv(
    filepath_or_buffer=dataPath / "plierRes_residual.csv", index_col=0
)
plierRes_u = pd.read_csv(filepath_or_buffer=dataPath / "plierRes_u.csv", index_col=0)
plierRes_z = pd.read_csv(filepath_or_buffer=dataPath / "plierRes_z.csv", index_col=0)
plierRes_heldoutgenes = pd.read_csv(
    filepath_or_buffer=dataPath / "plierRes_heldOutGenes.csv", index_col=0
)

plierRes_up = pd.read_csv(filepath_or_buffer=dataPath / "plierRes_up.csv", index_col=0)
plierRes_uauc = pd.read_csv(
    filepath_or_buffer=dataPath / "plierRes_uauc.csv", index_col=0
)
plierRes_summary = pd.read_csv(
    filepath_or_buffer=dataPath / "plierRes_summary.csv", index_col=0
)

priorMat = pd.read_csv(
    filepath_or_buffer=dataPath / "plierRes_priorMat.csv", index_col=0
)
pathwayMat = pd.read_csv(
    filepath_or_buffer=dataPath / "plierRes_pathwayMat.csv", index_col=0
)

In [179]:
plierRes = {
    "residual": plierRes_residual,
    "B": plierRes_b,
    "C": plierRes_c,
    "U": plierRes_u,
    "Z": plierRes_z,
    "L1": 18.5633,
    "L2": 37.12661,
    "L3": 0.0005530844,
    "Up": plierRes_up,
    "Uauc": plierRes_uauc,
    "summary": plierRes_summary,
    "heldOutGenes": {
        k: g["value"].tolist() for k, g in plierRes_heldoutgenes.groupby("name")
    },
}

In [180]:
ngenes = 50
auc_cutoff = 0.2
fdr_cutoff = 0.01

In [181]:
data: pd.DataFrame = dataWholeBlood
priorMat: pd.DataFrame = allPaths
svdres = None
num_LVs = None
L1 = None
L2 = None
L3 = None
frac: float = 0.7
max_iter: int = 350
trace = False
scale = True
Chat = None
maxPath: int = 10
doCrossval: bool = True
penalty_factor = None
glm_alpha: float = 0.9
minGenes: int = 10
tol: float = 1e-06
seed: int = 123456
allGenes: bool = False
rseed = None
pathwaySelection: str = "complete"

In [182]:
import importlib.util

spec = importlib.util.spec_from_file_location(
    "pyplier.utils", "/Users/milessmith/workspace/pyplier/src/pyplier/utils.py"
)
utils = importlib.util.module_from_spec(spec)
spec.loader.exec_module(utils)

In [183]:
import importlib.util

spec = importlib.util.spec_from_file_location(
    "pyplier", "/Users/milessmith/workspace/pyplier/src/pyplier/__init__.py"
)
pyplier = importlib.util.module_from_spec(spec)
spec.loader.exec_module(pyplier)

In [184]:
if penalty_factor is None:
    penalty_factor = range(priorMat.shape[1])

if scale:
    Y = pyplier.rowNorm(data)
else:
    Y = data

In [185]:
if (priorMat.shape[0] != data.shape[0]) or not all(priorMat.index == data.index):
    if not allGenes:
        cm = pyplier.commonRows(data, priorMat)
        print(f"Selecting common genes: {len(cm)}")
        priorMat = priorMat.loc[cm, :]
        Y = Y.loc[cm, :]
    else:
        extra_genes = pyplier.setdiff(data.index, priorMat.index)
        eMat = pd.DataFrame(
            data=np.zeros((len(extra_genes), priorMat.shape[1])),
            columns=priorMat.columns,
            index=extra_genes,
        )
        priorMat = priorMat.append(eMat)
        priorMat = priorMat.loc[data.index, :]

Selecting common genes: 5892

In [186]:
numGenes = priorMat.sum(axis="rows")  # colsums

heldOutGenes = dict()
iibad = numGenes[numGenes < minGenes].index
priorMat.loc[:, iibad] = 0
print(f"Removing {len(iibad)} pathways with too few genes")

Removing 3 pathways with too few genes

In [187]:
if doCrossval:
    priorMatCV = priorMat
    if seed is not None:
        random.seed(seed)
    for j in range(priorMatCV.shape[1]):
        current_col = priorMatCV.iloc[:, j]
        iipos = [
            current_col.index.get_loc(_) for _ in current_col[current_col > 0].index
        ]  # need the row number, not the row name

        iiposs = random.choices(iipos, k=floor(len(iipos) / 5))
        priorMatCV.iloc[iiposs, j] = 0
        heldOutGenes[priorMat.columns[j]] = priorMat.index[iiposs].values
    C = priorMatCV
else:
    C = priorMat

In [188]:
ns = data.shape[1]
Bdiff = -1
BdiffTrace = np.ndarray((0,), dtype=np.float64)
BdiffCount = 0
if Chat is None:
    Cp = pyplier.crossprod(C)
    Chat = pyplier.pinv_ridge(Cp, 5) @ C.transpose()

In [189]:
if (svdres is not None) and (svdres["v"] != Y.shape[1]):
    print("SVD V has the wrong number of columns")
    svdres = None

In [190]:
if Y.isnull().to_numpy().any():
    Y.fillna(0, inplace=True)

In [191]:
svdres = None

In [192]:
svdres is None

True

In [193]:
from scipy.linalg import svd

In [194]:
if svdres is None:
    svdres = dict()
    print("Computing SVD")
    if ns > 500:
        print("Using rsvd")
        svdres["u"], svdres["d"], svdres["v"] = randomized_svd(
            M=Y.values, n_components=min(ns, max(200, ns / 4)), n_iter=3
        )
    else:
        svdres["u"], svdres["d"], svdres["v"] = svd(Y, lapack_driver="gesdd")
        svdres["v"] = svdres["v"].transpose()
    print("Done")

Computing SVD

Done

In [265]:
svdres["u"], svdres["d"], svdres["v"] = svd(Y, lapack_driver="gesdd")

In [196]:
from numpy.linalg import svd as nsvd

In [199]:
rsvdres = dict()
rsvdres["u"], rsvdres["d"], rsvdres["v"] = randomized_svd(
    M=Y.values, n_components=min(ns, max(200, ns / 4)), n_iter=3
)

/Users/milessmith/Library/Caches/pypoetry/virtualenvs/pyplier-JrjK94hZ-py3.8/lib/python3.8/site-packages/sklearn/utils/extmath.py:368: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  warnings.warn(


In [200]:
pd.DataFrame(svdres["v"].T)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.177491,-0.030616,-0.117442,-0.290516,0.091422,-0.231867,-0.304756,0.151708,-0.045222,0.074408,...,-0.065073,0.090172,-0.090283,0.077909,0.056640,-0.009434,0.180497,0.049449,-0.165677,0.167650
1,0.093049,-0.184942,-0.158544,-0.009213,0.175392,0.091768,-0.202924,-0.103139,0.183736,-0.007996,...,0.382133,-0.047747,-0.135802,-0.493973,-0.085153,-0.056028,-0.356024,0.157118,-0.165677,0.167650
2,0.219910,0.123374,0.098651,-0.466178,-0.191182,-0.178881,0.510587,-0.410859,-0.171887,-0.082395,...,0.007576,-0.023934,0.027290,-0.025026,-0.003031,-0.008828,-0.072692,0.024175,-0.165677,0.167650
3,0.315039,0.281003,-0.268667,0.169794,-0.235057,-0.171600,-0.046893,0.054007,-0.074697,0.049759,...,-0.089474,-0.040158,0.064712,-0.052188,0.102649,-0.068409,0.088942,-0.071505,-0.165677,0.167650
4,0.045651,0.053069,-0.043128,0.199062,0.041016,0.026442,0.014069,-0.173721,0.259488,0.179553,...,0.038517,0.195320,-0.018134,0.159408,0.408517,0.361339,-0.090716,0.007183,-0.165677,0.167650
5,0.176812,-0.037460,0.023752,0.118690,0.100901,0.061268,-0.166273,-0.181984,0.091609,0.177258,...,-0.178735,0.077924,0.216092,0.122121,-0.495273,0.091949,0.297833,-0.118791,-0.165677,0.167650
6,-0.178758,-0.357693,-0.142183,0.064649,0.173731,0.118532,0.197576,-0.133709,-0.028311,-0.106049,...,-0.207855,0.129233,-0.203192,0.254934,0.003012,-0.068940,0.083106,0.003171,-0.165677,0.167650
7,-0.016757,-0.047299,-0.128663,0.179993,0.116109,0.004497,0.029049,-0.200074,-0.019563,-0.259006,...,0.099588,-0.035138,0.284058,-0.154448,0.017127,0.083266,0.320843,-0.052010,-0.165677,0.167650
8,0.123139,-0.254089,-0.042924,-0.203351,-0.133235,0.129448,0.125710,0.166770,0.354800,0.118921,...,-0.229437,-0.288986,0.106502,0.020896,0.202660,0.001181,-0.186949,-0.047172,-0.165677,0.167650
9,-0.046432,-0.126237,-0.113823,-0.042520,0.169603,-0.076227,0.376079,0.515754,-0.193513,0.279912,...,0.139760,0.030296,0.076010,-0.001690,-0.015110,-0.052359,-0.004798,0.011698,-0.165677,0.167650


In [268]:
svdres["v"] = svdres["v"].T

In [202]:
if num_LVs is None:
    num_LVs = pyplier.num_pc(svdres) * 2
    num_LVs = min(num_LVs, floor(Y.shape[1] * 0.9))
    print(f"The number of LVs is set to {num_LVs}")

Smoothing data

The number of LVs is set to 30

In [203]:
L2 = None

In [204]:
if L2 is None:
    L2 = svdres["d"][num_LVs - 1]
    print(f"L2 is set to {L2}")

L2 is set to 36.86116786718935

In [205]:
if L1 is None:
    L1 = L2 / 2
    print(f"L1 is set to {L1}")

L1 is set to 18.430583933594676

In [269]:
B = (svdres["v"][: Y.shape[1], :num_LVs] @ np.diag(svdres["d"][:num_LVs])).transpose()

In [270]:
pd.DataFrame(B)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,43.038151,22.562642,53.323955,76.390763,11.069577,42.873306,-43.345225,-4.063175,29.858852,-11.258819,...,32.288634,13.688157,35.439813,46.731125,20.587638,0.816835,-0.872124,-23.785007,-7.991206,-73.135317
1,-5.953241,-35.961385,23.989670,54.640167,10.319179,-7.283881,-69.552155,-9.197142,-49.406754,-24.546452,...,67.843182,-21.393124,-17.312752,-61.842963,27.852610,-48.505408,6.963699,-0.284867,-18.259852,-11.725350
2,-16.293202,-21.995528,13.686335,-37.273319,-5.983304,3.295214,-19.725663,-17.849959,-5.955057,-15.791134,...,-13.379106,-65.064560,12.432474,32.820164,37.241073,22.699675,11.581313,23.755779,-16.389390,-19.461863
3,-27.974824,-0.887174,-44.890009,16.350100,19.168408,11.429069,6.225307,17.332229,-19.581418,-4.094389,...,16.648865,-30.563128,12.428113,20.396534,-16.682882,-8.079198,-12.333890,0.747644,-6.697438,10.982125
4,7.864873,15.088596,-16.447021,-20.221474,3.528512,8.680293,14.945685,9.988601,-11.461937,14.590559,...,23.450625,15.284928,-17.538423,-13.693015,16.285523,7.036109,-6.976071,19.845310,9.374499,0.730080
5,-19.698073,7.796068,-15.196754,-14.578152,2.246352,5.204972,10.069795,0.382070,10.997131,-6.475850,...,2.668860,21.452075,6.551688,8.848991,14.964228,-4.417374,17.973018,-25.305732,-12.342067,-32.050671
6,23.445979,15.611682,-39.281250,3.607671,-1.082400,12.792002,-15.200235,-2.234849,-9.671333,-28.933093,...,-17.256532,9.171563,-11.435294,-9.130000,-7.402119,5.445279,14.282574,4.240444,-4.991241,-6.650379
7,-10.914391,7.420156,29.558668,-3.885464,12.498104,13.092566,9.619495,14.394035,-11.998017,-37.105139,...,-22.654199,1.892581,5.338905,-5.842165,-8.803908,-3.281052,-0.716909,7.808142,9.106194,-0.482875
8,2.990115,-12.148869,11.365372,4.939044,-17.157698,-6.057330,1.871958,1.293505,-23.459829,12.795312,...,4.693580,8.339831,-3.086462,2.238423,-2.019101,-13.559567,-3.045497,8.170529,-7.386461,-7.575866
9,-4.729696,0.508238,5.237372,-3.162908,-11.413161,-11.267329,6.740950,16.463585,-7.559155,-17.792443,...,13.476641,3.507187,0.973766,-4.031216,-1.915725,13.964474,-7.704302,-0.319509,-2.813099,-3.077001


In [271]:
tmp = np.zeros((6, 6))
np.fill_diagonal(tmp, 1)
tmp

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [272]:
diag_mat = np.zeros((num_LVs, num_LVs))
np.fill_diagonal(diag_mat, val=1)

In [273]:
pyplier.tcrossprod(B).shape
(L1 * diag_mat).shape

(30, 30)

(30, 30)

In [276]:
Z = (Y @ B.transpose()) @ solve((np.dot(B, B.T) + L1 * diag_mat), diag_mat)

In [277]:
Z

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
gene,,,,,,,,,,,,,,,,,,,,,
GAS6,-0.012745,-0.001176,0.003765,0.004943,0.006964,-0.005372,-0.027436,0.030357,0.014092,0.017775,...,0.016648,-0.019022,-0.018361,-0.000755,0.014004,-0.015134,-0.013787,0.026021,0.012807,0.001198
MMP14,-0.003188,-0.007476,0.000563,0.018835,0.018096,-0.016484,-0.022289,0.005958,0.012575,-0.000004,...,0.004161,-0.013513,0.002308,0.028636,0.012690,-0.050614,-0.048964,0.004334,-0.030275,-0.022011
MARCKSL1,-0.015076,-0.013287,-0.015628,0.003894,-0.002163,0.006716,0.006762,0.003081,-0.021827,-0.013620,...,-0.014620,-0.007819,-0.016893,0.037771,0.006202,-0.002661,0.000248,-0.008354,0.009328,-0.007988
SPARC,-0.003257,-0.000186,-0.004765,-0.012698,-0.052616,-0.010515,0.017722,0.008765,-0.003268,-0.017309,...,0.003166,0.011693,0.000154,-0.022788,0.020946,0.009999,0.006880,0.000625,-0.004172,0.006067
CTSD,-0.018330,-0.007472,-0.016026,0.005452,0.012843,-0.018580,-0.000105,0.000467,0.002519,-0.015562,...,0.004841,0.010316,0.005150,-0.000026,0.007914,-0.000590,-0.001699,0.004089,-0.007631,-0.005490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CFL2,0.011959,-0.006863,-0.004746,-0.012191,0.022164,0.011292,-0.007798,-0.010608,0.000363,-0.017646,...,0.010743,0.020460,-0.004573,-0.017192,-0.024710,-0.015373,-0.043126,0.012508,0.003710,0.007713
CFL1,-0.008415,-0.019710,-0.009325,0.015125,0.009008,-0.010146,0.011763,-0.008494,0.000695,0.004028,...,0.001943,0.006358,-0.033747,-0.010074,-0.001326,-0.004542,-0.023237,-0.010814,-0.009072,-0.014627
SELL,0.006495,0.023573,-0.006146,0.008217,0.002978,0.004064,-0.015178,-0.014239,-0.009563,-0.000312,...,-0.006695,0.009901,-0.002953,-0.006853,0.008605,-0.006094,0.022384,0.005652,-0.003739,0.023476


In [279]:
Zprime = Z.copy()
Zprime[Zprime < 0] = 0
Zprime

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
gene,,,,,,,,,,,,,,,,,,,,,
GAS6,0.000000,0.000000,0.003765,0.004943,0.006964,0.000000,0.000000,0.030357,0.014092,0.017775,...,0.016648,0.000000,0.000000,0.000000,0.014004,0.000000,0.000000,0.026021,0.012807,0.001198
MMP14,0.000000,0.000000,0.000563,0.018835,0.018096,0.000000,0.000000,0.005958,0.012575,0.000000,...,0.004161,0.000000,0.002308,0.028636,0.012690,0.000000,0.000000,0.004334,0.000000,0.000000
MARCKSL1,0.000000,0.000000,0.000000,0.003894,0.000000,0.006716,0.006762,0.003081,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.037771,0.006202,0.000000,0.000248,0.000000,0.009328,0.000000
SPARC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017722,0.008765,0.000000,0.000000,...,0.003166,0.011693,0.000154,0.000000,0.020946,0.009999,0.006880,0.000625,0.000000,0.006067
CTSD,0.000000,0.000000,0.000000,0.005452,0.012843,0.000000,0.000000,0.000467,0.002519,0.000000,...,0.004841,0.010316,0.005150,0.000000,0.007914,0.000000,0.000000,0.004089,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CFL2,0.011959,0.000000,0.000000,0.000000,0.022164,0.011292,0.000000,0.000000,0.000363,0.000000,...,0.010743,0.020460,0.000000,0.000000,0.000000,0.000000,0.000000,0.012508,0.003710,0.007713
CFL1,0.000000,0.000000,0.000000,0.015125,0.009008,0.000000,0.011763,0.000000,0.000695,0.004028,...,0.001943,0.006358,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
SELL,0.006495,0.023573,0.000000,0.008217,0.002978,0.004064,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.009901,0.000000,0.000000,0.008605,0.000000,0.022384,0.005652,0.000000,0.023476


In [284]:
Z.where(cond=lambda x: x > 0, other=0, inplace=True)

In [285]:
Z.loc[["EPAS1"], :]

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
gene,,,,,,,,,,,,,,,,,,,,,
EPAS1,0.003014,0.016512,0.0,0.004984,0.0,0.008747,0.011687,0.03117,0.012553,0.0,...,0.004864,0.032634,0.006588,0.0,0.0,0.001753,0.001561,0.0,0.015823,0.012357


In [215]:
alfa = np.reshape(range(36), (6, 6))

In [216]:
alfa

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35]])

In [217]:
from numpy.random import default_rng

rng = default_rng()

In [218]:
rng.shuffle(alfa, axis=1)

In [219]:
alfa.T

array([[ 1,  7, 13, 19, 25, 31],
       [ 5, 11, 17, 23, 29, 35],
       [ 3,  9, 15, 21, 27, 33],
       [ 2,  8, 14, 20, 26, 32],
       [ 0,  6, 12, 18, 24, 30],
       [ 4, 10, 16, 22, 28, 34]])

In [220]:
U = np.zeros((C.shape[1], num_LVs))

In [286]:
(
    Y - (Z @ B).rename(columns={k: v for k, v in enumerate(Y.columns)}) ** 2
).to_numpy().mean()

-0.4725963959256272

In [222]:
Y.shape
Z.shape
B.shape

(5892, 36)

(5892, 30)

(30, 36)

In [287]:
(Z @ B)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
gene,,,,,,,,,,,,,,,,,,,,,
GAS6,-0.358844,0.352292,0.761736,-0.089370,0.595133,-0.011760,1.501955,0.772162,-1.605172,-1.045748,...,-0.169690,-0.354437,0.347460,-0.335907,0.302990,0.120353,-0.876549,0.268926,-0.760225,-1.038132
MMP14,-0.749488,0.564777,-0.756180,0.060636,0.351472,0.916624,0.329737,0.137899,-1.311097,1.131892,...,-0.083303,-0.659989,0.186013,-0.132781,-0.324325,-0.242005,-0.817507,0.652026,0.007453,-0.503421
MARCKSL1,-0.581996,0.510506,-0.545605,0.359963,-0.187277,0.481525,0.195848,-0.103896,0.171351,-0.296986,...,-0.176404,-0.083556,0.129132,0.104605,0.047374,-0.244319,-0.636941,-0.108701,-0.023012,-0.393588
SPARC,0.255376,0.178495,-0.429140,0.080234,0.446638,0.576311,0.126836,-0.294160,-0.312225,-0.607212,...,-0.745006,-0.020282,-0.176802,-0.494510,0.297649,0.384150,-0.199912,-0.151883,-0.275142,-0.278330
CTSD,0.090215,0.225007,-0.329917,-0.315563,0.159687,0.518152,0.094908,0.227146,-0.590264,0.738175,...,0.290699,-0.160402,0.060802,-0.129086,-0.220147,0.434813,-0.058938,0.377381,-0.334518,-0.263651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CFL2,0.209608,0.560989,-0.080145,0.464964,0.208882,1.153630,0.150300,0.121737,0.351706,0.553511,...,0.683774,0.934549,0.240661,0.252721,0.964606,0.304173,-0.252609,0.451716,-1.106011,-1.512154
CFL1,0.167582,0.491587,-1.159473,0.250858,0.232732,0.335903,0.087708,0.417319,-1.031843,-0.014909,...,-0.037669,-0.411436,-0.223956,0.079917,-0.096231,0.594047,-0.139514,0.145222,-0.285152,-0.134444
SELL,-0.065740,-1.457501,0.359508,1.775592,0.562748,0.523200,-1.174680,-0.025679,-0.743925,-0.745162,...,1.983427,-0.672172,-0.286410,-1.155735,0.844468,-1.055409,0.132165,-0.275024,-1.003050,-1.022071


In [289]:
pd.DataFrame(np.dot(Z, B))

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,-0.358844,0.352292,0.761736,-0.089370,0.595133,-0.011760,1.501955,0.772162,-1.605172,-1.045748,...,-0.169690,-0.354437,0.347460,-0.335907,0.302990,0.120353,-0.876549,0.268926,-0.760225,-1.038132
1,-0.749488,0.564777,-0.756180,0.060636,0.351472,0.916624,0.329737,0.137899,-1.311097,1.131892,...,-0.083303,-0.659989,0.186013,-0.132781,-0.324325,-0.242005,-0.817507,0.652026,0.007453,-0.503421
2,-0.581996,0.510506,-0.545605,0.359963,-0.187277,0.481525,0.195848,-0.103896,0.171351,-0.296986,...,-0.176404,-0.083556,0.129132,0.104605,0.047374,-0.244319,-0.636941,-0.108701,-0.023012,-0.393588
3,0.255376,0.178495,-0.429140,0.080234,0.446638,0.576311,0.126836,-0.294160,-0.312225,-0.607212,...,-0.745006,-0.020282,-0.176802,-0.494510,0.297649,0.384150,-0.199912,-0.151883,-0.275142,-0.278330
4,0.090215,0.225007,-0.329917,-0.315563,0.159687,0.518152,0.094908,0.227146,-0.590264,0.738175,...,0.290699,-0.160402,0.060802,-0.129086,-0.220147,0.434813,-0.058938,0.377381,-0.334518,-0.263651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5887,0.209608,0.560989,-0.080145,0.464964,0.208882,1.153630,0.150300,0.121737,0.351706,0.553511,...,0.683774,0.934549,0.240661,0.252721,0.964606,0.304173,-0.252609,0.451716,-1.106011,-1.512154
5888,0.167582,0.491587,-1.159473,0.250858,0.232732,0.335903,0.087708,0.417319,-1.031843,-0.014909,...,-0.037669,-0.411436,-0.223956,0.079917,-0.096231,0.594047,-0.139514,0.145222,-0.285152,-0.134444
5889,-0.065740,-1.457501,0.359508,1.775592,0.562748,0.523200,-1.174680,-0.025679,-0.743925,-0.745162,...,1.983427,-0.672172,-0.286410,-1.155735,0.844468,-1.055409,0.132165,-0.275024,-1.003050,-1.022071
5890,0.228721,0.313158,-1.227452,0.150025,0.020481,0.817893,0.230289,0.148920,-1.124992,0.593031,...,0.137840,0.409227,-0.825082,-0.785767,0.367812,0.002887,0.008176,0.829116,-0.601764,-0.601661


In [227]:
Y

,BD8001,BD8002,BD8003,BD8004,BD8005,BD8006,BD8007,BD8008,BD8009,BD8010,...,BD8031,BD8032,BD8033,BD8034,BD8038,BD8041,BD8042,BD8043,BD8044,BD8045
gene,,,,,,,,,,,,,,,,,,,,,
GAS6,-1.505242,0.188428,1.382268,-0.886040,0.218978,-1.659888,2.796534,0.829210,-1.649642,-0.699206,...,0.167190,-0.443537,-0.413695,-0.033962,-0.003753,-0.562015,-1.457108,0.254652,-0.945488,1.285548
MMP14,-1.362254,1.426659,-0.112956,-0.547275,-0.657120,0.623815,0.162686,0.399899,-0.959655,2.260257,...,-0.136098,-0.776283,0.401661,-0.029038,-1.121673,-0.977343,-1.771282,0.948462,0.723387,0.895571
MARCKSL1,-1.559936,1.247540,-2.110556,-0.721440,-0.058441,0.154611,1.487958,-0.045567,1.409842,0.235813,...,-2.010731,1.165882,-0.192181,-0.207469,-1.077898,0.035008,-0.486909,-0.433482,0.683989,2.203569
SPARC,0.465116,-0.849214,0.790658,0.906851,0.163648,0.117905,-1.060804,-1.701798,0.920468,-1.141010,...,-2.644481,-0.004298,0.902712,-0.857343,-0.450150,-0.669798,0.684989,-0.941961,-0.446238,0.891104
CTSD,0.024942,0.206589,-1.452483,-1.102910,0.034775,-0.016322,0.844618,0.483747,-0.650142,1.753523,...,-0.776214,0.643598,-0.739288,-0.768372,-1.777247,0.343815,-0.374175,0.949864,0.449915,2.400907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CFL2,1.024350,0.563739,-0.149946,0.020848,-0.585605,1.365620,0.057274,0.884602,0.876396,1.450731,...,0.163680,2.526496,0.364268,0.539036,1.949757,-0.305034,-0.267821,0.057269,-1.149261,-0.583529
CFL1,0.138011,1.291487,-1.981167,-1.081285,-0.753500,-0.460054,0.899439,1.291487,-0.358317,1.045690,...,-1.794891,1.080968,-0.282019,1.021460,-1.325723,1.361414,0.051768,0.379537,-0.130555,2.023380
SELL,-0.149550,-2.111934,0.225171,1.969528,0.393432,0.225171,-1.251822,0.225171,0.356145,-0.580925,...,2.896014,0.111915,-0.275184,-0.796417,1.273054,-1.280267,-0.314888,-0.865101,-0.897640,0.111915


In [291]:
Y - np.dot(Z, B)

,BD8001,BD8002,BD8003,BD8004,BD8005,BD8006,BD8007,BD8008,BD8009,BD8010,...,BD8031,BD8032,BD8033,BD8034,BD8038,BD8041,BD8042,BD8043,BD8044,BD8045
gene,,,,,,,,,,,,,,,,,,,,,
GAS6,-1.146398,-0.163864,0.620532,-0.796669,-0.376155,-1.648128,1.294580,0.057048,-0.044470,0.346541,...,0.336881,-0.089100,-0.761155,0.301945,-0.306743,-0.682368,-0.580559,-0.014274,-0.185263,2.323681
MMP14,-0.612766,0.861881,0.643223,-0.607911,-1.008592,-0.292809,-0.167051,0.262001,0.351443,1.128365,...,-0.052795,-0.116294,0.215648,0.103743,-0.797348,-0.735338,-0.953775,0.296436,0.715934,1.398992
MARCKSL1,-0.977940,0.737034,-1.564951,-1.081403,0.128836,-0.326914,1.292110,0.058329,1.238491,0.532799,...,-1.834326,1.249438,-0.321313,-0.312073,-1.125272,0.279327,0.150032,-0.324781,0.707001,2.597156
SPARC,0.209740,-1.027708,1.219799,0.826617,-0.282989,-0.458406,-1.187641,-1.407638,1.232693,-0.533797,...,-1.899475,0.015984,1.079514,-0.362833,-0.747799,-1.053948,0.884901,-0.790078,-0.171096,1.169434
CTSD,-0.065273,-0.018418,-1.122566,-0.787347,-0.124912,-0.534474,0.749710,0.256600,-0.059878,1.015348,...,-1.066913,0.804000,-0.800090,-0.639286,-1.557101,-0.090998,-0.315238,0.572483,0.784434,2.664557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CFL2,0.814742,0.002750,-0.069802,-0.444116,-0.794487,0.211990,-0.093026,0.762865,0.524690,0.897221,...,-0.520094,1.591947,0.123607,0.286315,0.985151,-0.609207,-0.015212,-0.394447,-0.043250,0.928625
CFL1,-0.029571,0.799900,-0.821695,-1.332143,-0.986231,-0.795957,0.811731,0.874168,0.673526,1.060598,...,-1.757222,1.492404,-0.058063,0.941543,-1.229492,0.767367,0.191282,0.234314,0.154597,2.157824
SELL,-0.083809,-0.654433,-0.134336,0.193936,-0.169316,-0.298029,-0.077143,0.250850,1.100069,0.164238,...,0.912587,0.784087,0.011225,0.359317,0.428586,-0.224859,-0.447053,-0.590077,0.105410,1.133986


In [292]:
(Y - np.dot(Z, B)) ** 2

,BD8001,BD8002,BD8003,BD8004,BD8005,BD8006,BD8007,BD8008,BD8009,BD8010,...,BD8031,BD8032,BD8033,BD8034,BD8038,BD8041,BD8042,BD8043,BD8044,BD8045
gene,,,,,,,,,,,,,,,,,,,,,
GAS6,1.314229,0.026851,0.385060,0.634682,0.141493,2.716326,1.675936,0.003254,0.001978,0.120091,...,0.113489,0.007939,0.579356,0.091171,0.094091,0.465627,0.337049,0.000204,0.034322,5.399492
MMP14,0.375482,0.742839,0.413736,0.369556,1.017259,0.085737,0.027906,0.068644,0.123512,1.273208,...,0.002787,0.013524,0.046504,0.010763,0.635763,0.540722,0.909686,0.087874,0.512562,1.957178
MARCKSL1,0.956367,0.543219,2.449071,1.169433,0.016599,0.106872,1.669549,0.003402,1.533859,0.283875,...,3.364752,1.561096,0.103242,0.097390,1.266238,0.078024,0.022509,0.105483,0.499850,6.745221
SPARC,0.043991,1.056185,1.487909,0.683295,0.080083,0.210136,1.410490,1.981446,1.519532,0.284940,...,3.608003,0.000255,1.165350,0.131648,0.559203,1.110807,0.783050,0.624223,0.029274,1.367576
CTSD,0.004261,0.000339,1.260155,0.619915,0.015603,0.285662,0.562065,0.065844,0.003585,1.030931,...,1.138304,0.646416,0.640144,0.408687,2.424563,0.008281,0.099375,0.327736,0.615336,7.099866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CFL2,0.663805,0.000008,0.004872,0.197239,0.631210,0.044940,0.008654,0.581963,0.275300,0.805005,...,0.270497,2.534294,0.015279,0.081976,0.970522,0.371133,0.000231,0.155588,0.001871,0.862344
CFL1,0.000874,0.639840,0.675182,1.774604,0.972652,0.633547,0.658908,0.764169,0.453637,1.124869,...,3.087829,2.227269,0.003371,0.886503,1.511650,0.588852,0.036589,0.054903,0.023900,4.656206
SELL,0.007024,0.428282,0.018046,0.037611,0.028668,0.088821,0.005951,0.062926,1.210153,0.026974,...,0.832816,0.614792,0.000126,0.129109,0.183686,0.050561,0.199856,0.348190,0.011111,1.285923


In [295]:
((Y - np.dot(Z, B)) ** 2).to_numpy().mean()

0.49708478198847605

In [297]:
print(
    f"errorY (SVD based:best possible) = {((Y - np.dot(Z, B))**2).to_numpy().mean():.4f}"
)

errorY (SVD based:best possible) = 0.4971

In [249]:
pd.DataFrame(solve(a=np.dot(B, B.T) + L1 * diag_mat, b=diag_mat))

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.700243e-05,-1.568737e-21,4.861752e-22,-1.051870e-20,4.110632e-21,1.115347e-21,-7.495739e-21,1.840222e-20,-9.970750e-23,2.987023e-21,...,3.120365e-23,3.084012e-21,-3.335693e-21,1.461159e-21,-5.962023e-21,-4.888953e-21,2.359656e-20,-5.602189e-21,7.641477e-21,-1.030810e-20
1,-1.568737e-21,2.643550e-05,1.639167e-22,-1.309851e-21,-1.637840e-20,-8.778998e-21,4.389166e-21,3.383201e-20,2.394412e-21,-1.871319e-21,...,1.253349e-20,4.794724e-21,4.479010e-21,-1.864128e-20,-3.441821e-20,7.874587e-21,1.175229e-20,-2.497957e-20,-1.169996e-20,-2.683100e-20
2,4.861752e-22,1.639167e-22,5.190576e-05,-5.270887e-20,-1.277244e-20,-8.749852e-21,-2.786727e-20,1.457850e-20,2.986760e-21,-3.071873e-20,...,1.349583e-20,2.403801e-21,-2.748725e-20,-2.728681e-20,3.418136e-21,-7.241273e-20,3.260917e-20,-1.047495e-20,2.759196e-20,8.122397e-21
3,-1.051870e-20,-1.309851e-21,-5.270887e-20,1.076320e-04,4.442859e-20,2.415645e-21,-6.075794e-20,2.726439e-20,3.396504e-21,1.770837e-20,...,2.872822e-20,-1.919052e-20,4.013049e-20,2.268004e-20,-3.664669e-20,7.597952e-21,7.904124e-21,4.250313e-20,-1.977929e-20,8.913025e-21
4,4.110632e-21,-1.637840e-20,-1.277244e-20,4.442859e-20,1.347847e-04,-1.023726e-19,-5.606106e-20,4.551901e-20,4.149120e-20,-1.576010e-20,...,4.440779e-20,-1.452747e-19,1.422678e-19,-6.261217e-21,-1.053583e-20,4.471483e-20,-5.826003e-20,2.791308e-21,1.732046e-20,4.406533e-20
5,1.115347e-21,-8.778998e-21,-8.749852e-21,2.415645e-21,-1.023726e-19,1.382042e-04,4.108920e-20,-7.334895e-20,8.752607e-21,-4.031026e-20,...,-9.816432e-20,4.462210e-21,-2.487340e-20,-3.510629e-20,2.661823e-20,3.531332e-20,1.011776e-20,6.748221e-20,5.264973e-20,-2.087964e-20
6,-7.495739e-21,4.389166e-21,-2.786727e-20,-6.075794e-20,-5.606106e-20,4.108920e-20,1.684294e-04,4.159446e-20,-2.306536e-20,-7.580522e-21,...,-1.355991e-20,-1.548300e-20,3.129298e-20,-1.078768e-20,1.060414e-19,-1.181163e-20,5.417961e-21,-5.730118e-20,6.131894e-20,5.445006e-20
7,1.840222e-20,3.383201e-20,1.457850e-20,2.726439e-20,4.551901e-20,-7.334895e-20,4.159446e-20,1.925188e-04,-7.106103e-20,-3.189564e-20,...,4.457236e-20,1.113456e-20,1.969781e-20,3.790871e-20,6.994671e-20,-3.752388e-20,4.505241e-20,4.762528e-20,1.277165e-20,3.756517e-20
8,-9.970750e-23,2.394412e-21,2.986760e-21,3.396504e-21,4.149120e-20,8.752607e-21,-2.306536e-20,-7.106103e-20,2.277664e-04,3.027781e-20,...,1.494831e-21,-3.016838e-20,3.621362e-20,-2.327186e-20,1.968293e-20,-1.776161e-20,1.575567e-20,1.869271e-20,7.386939e-21,4.918152e-20
9,2.987023e-21,-1.871319e-21,-3.071873e-20,1.770837e-20,-1.576010e-20,-4.031026e-20,-7.580522e-21,-3.189564e-20,3.027781e-20,2.463743e-04,...,9.990163e-22,3.533495e-20,6.123160e-20,9.035935e-20,-3.313344e-21,2.668195e-20,4.053856e-21,-2.570991e-20,1.077760e-19,-3.841648e-20


In [231]:
pd.DataFrame(solve(a=np.dot(B, B.T) + L1 * diag_mat, b=diag_mat))

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.700243e-05,-1.568737e-21,4.861752e-22,-1.051870e-20,4.110632e-21,1.115347e-21,-7.495739e-21,1.840222e-20,-9.970750e-23,2.987023e-21,...,3.120365e-23,3.084012e-21,-3.335693e-21,1.461159e-21,-5.962023e-21,-4.888953e-21,2.359656e-20,-5.602189e-21,7.641477e-21,-1.030810e-20
1,-1.568737e-21,2.643550e-05,1.639167e-22,-1.309851e-21,-1.637840e-20,-8.778998e-21,4.389166e-21,3.383201e-20,2.394412e-21,-1.871319e-21,...,1.253349e-20,4.794724e-21,4.479010e-21,-1.864128e-20,-3.441821e-20,7.874587e-21,1.175229e-20,-2.497957e-20,-1.169996e-20,-2.683100e-20
2,4.861752e-22,1.639167e-22,5.190576e-05,-5.270887e-20,-1.277244e-20,-8.749852e-21,-2.786727e-20,1.457850e-20,2.986760e-21,-3.071873e-20,...,1.349583e-20,2.403801e-21,-2.748725e-20,-2.728681e-20,3.418136e-21,-7.241273e-20,3.260917e-20,-1.047495e-20,2.759196e-20,8.122397e-21
3,-1.051870e-20,-1.309851e-21,-5.270887e-20,1.076320e-04,4.442859e-20,2.415645e-21,-6.075794e-20,2.726439e-20,3.396504e-21,1.770837e-20,...,2.872822e-20,-1.919052e-20,4.013049e-20,2.268004e-20,-3.664669e-20,7.597952e-21,7.904124e-21,4.250313e-20,-1.977929e-20,8.913025e-21
4,4.110632e-21,-1.637840e-20,-1.277244e-20,4.442859e-20,1.347847e-04,-1.023726e-19,-5.606106e-20,4.551901e-20,4.149120e-20,-1.576010e-20,...,4.440779e-20,-1.452747e-19,1.422678e-19,-6.261217e-21,-1.053583e-20,4.471483e-20,-5.826003e-20,2.791308e-21,1.732046e-20,4.406533e-20
5,1.115347e-21,-8.778998e-21,-8.749852e-21,2.415645e-21,-1.023726e-19,1.382042e-04,4.108920e-20,-7.334895e-20,8.752607e-21,-4.031026e-20,...,-9.816432e-20,4.462210e-21,-2.487340e-20,-3.510629e-20,2.661823e-20,3.531332e-20,1.011776e-20,6.748221e-20,5.264973e-20,-2.087964e-20
6,-7.495739e-21,4.389166e-21,-2.786727e-20,-6.075794e-20,-5.606106e-20,4.108920e-20,1.684294e-04,4.159446e-20,-2.306536e-20,-7.580522e-21,...,-1.355991e-20,-1.548300e-20,3.129298e-20,-1.078768e-20,1.060414e-19,-1.181163e-20,5.417961e-21,-5.730118e-20,6.131894e-20,5.445006e-20
7,1.840222e-20,3.383201e-20,1.457850e-20,2.726439e-20,4.551901e-20,-7.334895e-20,4.159446e-20,1.925188e-04,-7.106103e-20,-3.189564e-20,...,4.457236e-20,1.113456e-20,1.969781e-20,3.790871e-20,6.994671e-20,-3.752388e-20,4.505241e-20,4.762528e-20,1.277165e-20,3.756517e-20
8,-9.970750e-23,2.394412e-21,2.986760e-21,3.396504e-21,4.149120e-20,8.752607e-21,-2.306536e-20,-7.106103e-20,2.277664e-04,3.027781e-20,...,1.494831e-21,-3.016838e-20,3.621362e-20,-2.327186e-20,1.968293e-20,-1.776161e-20,1.575567e-20,1.869271e-20,7.386939e-21,4.918152e-20
9,2.987023e-21,-1.871319e-21,-3.071873e-20,1.770837e-20,-1.576010e-20,-4.031026e-20,-7.580522e-21,-3.189564e-20,3.027781e-20,2.463743e-04,...,9.990163e-22,3.533495e-20,6.123160e-20,9.035935e-20,-3.313344e-21,2.668195e-20,4.053856e-21,-2.570991e-20,1.077760e-19,-3.841648e-20


In [232]:
pd.DataFrame(np.dot(Y, B.T))

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-749.575191,-44.469007,72.534156,45.920696,51.670545,-38.866773,162.890212,-157.681222,-61.871163,-72.145354,...,-32.570558,-33.883795,32.671660,1.257162,22.356530,23.393667,20.609230,38.127579,-18.123070,1.650388
1,-187.505812,-282.790710,10.847584,174.994467,134.255394,-119.274871,132.332491,-30.949354,-55.210558,0.017612,...,-8.140121,-24.071884,-4.106445,-47.693372,20.257992,78.237019,73.191537,6.350173,42.840953,-30.312898
2,-886.702491,-502.600765,-301.082542,36.174370,-16.045575,48.592850,-40.147066,-16.003349,95.832246,55.281445,...,28.602917,-13.927702,30.059849,-62.908241,9.901566,4.113446,-0.370249,-12.240184,-13.199667,-11.000740
3,-191.554233,-7.051480,-91.800090,-117.980295,-390.372875,-76.085610,-105.219521,-45.526592,14.347406,70.255164,...,-6.194003,20.829141,-0.273694,37.953856,33.437272,-15.455990,-10.284977,0.915311,5.903577,8.354663
4,-1078.093719,-282.639988,-308.760925,50.653193,95.282256,-134.438826,0.623096,-2.423944,-11.057699,63.164839,...,-9.470382,18.375953,-9.164180,0.042723,12.634325,0.911879,2.540197,5.992032,10.797929,-7.560978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5887,703.371111,-259.595462,-91.434080,-113.267853,164.442433,81.705133,46.299802,55.099697,-1.594724,71.622336,...,-21.017346,36.446819,8.137330,28.633881,-39.446544,23.763623,64.465495,18.327802,-5.250098,10.622080
5888,-494.918972,-745.592763,-179.656810,140.529518,66.836121,-73.412576,-69.839180,44.120182,-3.052485,-16.349992,...,-3.802080,11.325475,60.050477,16.778061,-2.116842,7.021025,34.735055,-15.846001,12.837030,-20.143442
5889,381.995407,891.709980,-118.405742,76.346970,22.096601,29.405857,90.114203,73.961851,41.987744,1.266352,...,13.098918,17.636328,5.254856,11.413323,13.737483,9.420455,-33.460725,8.281207,5.290428,32.331152
5890,-402.070030,-576.019346,-146.102666,30.130400,163.483036,-47.192873,-124.535455,93.851176,-130.091976,3.587804,...,24.372402,-42.471422,-37.228559,4.865836,-40.056468,-2.687184,-20.677045,-15.543770,11.014733,-15.673465


In [233]:
Z = pd.DataFrame(
    np.dot(np.dot(Y, B.T), solve(a=np.dot(B, B.T) + L1 * diag_mat, b=diag_mat))
)

Z.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-0.012745,-0.001176,0.003765,0.004943,0.006964,-0.005372,0.027436,-0.030357,-0.014092,-0.017775,...,-0.016648,-0.019022,0.018361,0.000755,0.014004,0.015134,0.013787,0.026021,-0.012807,0.001198
1,-0.003188,-0.007476,0.000563,0.018835,0.018096,-0.016484,0.022289,-0.005958,-0.012575,0.000004,...,-0.004161,-0.013513,-0.002308,-0.028636,0.012690,0.050614,0.048964,0.004334,0.030275,-0.022011
2,-0.015076,-0.013287,-0.015628,0.003894,-0.002163,0.006716,-0.006762,-0.003081,0.021827,0.013620,...,0.014620,-0.007819,0.016893,-0.037771,0.006202,0.002661,-0.000248,-0.008354,-0.009328,-0.007988
3,-0.003257,-0.000186,-0.004765,-0.012698,-0.052616,-0.010515,-0.017722,-0.008765,0.003268,0.017309,...,-0.003166,0.011693,-0.000154,0.022788,0.020946,-0.009999,-0.006880,0.000625,0.004172,0.006067
4,-0.018330,-0.007472,-0.016026,0.005452,0.012843,-0.018580,0.000105,-0.000467,-0.002519,0.015562,...,-0.004841,0.010316,-0.005150,0.000026,0.007914,0.000590,0.001699,0.004089,0.007631,-0.005490


In [234]:
Z.fillna(0, inplace=True)

In [239]:
(
    (Y - (np.dot(Z, B))).rename(columns={k: v for k, v in enumerate(Y.columns)}) ** 2
).to_numpy().mean()

0.022388485114148926

In [240]:
(Y - (np.dot(Z, B)))

,BD8001,BD8002,BD8003,BD8004,BD8005,BD8006,BD8007,BD8008,BD8009,BD8010,...,BD8031,BD8032,BD8033,BD8034,BD8038,BD8041,BD8042,BD8043,BD8044,BD8045
gene,,,,,,,,,,,,,,,,,,,,,
GAS6,-0.030609,0.171123,0.043282,0.014176,0.036071,-0.364244,0.013223,-0.158560,0.117040,0.017348,...,0.020669,-0.043497,-0.088727,0.135188,-0.171217,-0.017344,0.083130,0.107015,-0.011600,-0.017794
MMP14,-0.201727,0.207695,0.030621,-0.133112,-0.295006,0.438257,-0.080672,-0.151080,-0.080578,0.009911,...,-0.001706,0.057403,0.087599,-0.528511,0.063641,0.003496,-0.187981,-0.114539,0.057392,-0.032027
MARCKSL1,-0.020161,0.020009,-0.000889,-0.039433,0.125428,0.045551,-0.033013,0.011308,-0.001272,-0.020880,...,-0.017259,-0.032839,-0.084566,0.046167,0.045413,0.020992,-0.094499,-0.005792,-0.066472,0.020078
SPARC,-0.084349,0.053445,0.011305,0.002185,-0.309169,0.176638,0.005251,-0.097062,-0.019730,0.020384,...,0.012676,0.091051,0.186026,-0.379858,-0.025984,-0.013561,0.033829,-0.055750,0.114353,-0.032423
CTSD,0.049047,-0.016684,-0.006259,0.014703,-0.114298,0.024659,0.027800,0.029813,-0.052633,0.020461,...,-0.019857,-0.017590,0.006904,0.175944,0.042724,-0.064658,0.064399,-0.047321,0.056799,0.005101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CFL2,0.093129,-0.496579,-0.115110,-0.038049,-0.242437,1.157505,0.015613,0.489664,-0.430755,-0.044807,...,-0.067746,0.163012,0.323175,-0.408406,0.561194,-0.007702,-0.254868,-0.377687,0.046802,0.055113
CFL1,0.104252,-0.189672,-0.037521,0.058912,0.061031,0.047635,0.029967,0.178052,-0.061067,-0.006743,...,-0.014049,0.018707,0.042020,0.127465,0.078060,0.002108,0.034725,-0.027396,-0.011969,0.030340
SELL,0.077806,-0.182189,-0.037065,0.020085,-0.017986,0.202849,0.030731,0.162439,-0.097169,-0.012216,...,-0.019237,0.017416,0.049236,0.074749,0.140736,-0.021111,-0.013852,-0.087559,0.008471,0.026381


In [241]:
pd.DataFrame(np.dot(Z, B))

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,-1.474633,0.017306,1.338986,-0.900216,0.182907,-1.295644,2.783311,0.987771,-1.766683,-0.716554,...,0.146522,-0.400040,-0.324968,-0.169151,0.167464,-0.544671,-1.540237,0.147637,-0.933888,1.303343
1,-1.160526,1.218963,-0.143577,-0.414163,-0.362114,0.185558,0.243358,0.550979,-0.879077,2.250346,...,-0.134391,-0.833685,0.314062,0.499473,-1.185313,-0.980839,-1.583301,1.063001,0.665995,0.927597
2,-1.539776,1.227532,-2.109667,-0.682006,-0.183870,0.109060,1.520971,-0.056876,1.411115,0.256693,...,-1.993472,1.198721,-0.107615,-0.253636,-1.123311,0.014016,-0.392410,-0.427691,0.750461,2.183491
3,0.549465,-0.902659,0.779353,0.904666,0.472817,-0.058734,-1.066056,-1.604736,0.940198,-1.161394,...,-2.657157,-0.095349,0.716686,-0.477486,-0.424166,-0.656237,0.651159,-0.886212,-0.560591,0.923528
4,-0.024105,0.223273,-1.446224,-1.117613,0.149073,-0.040981,0.816818,0.453934,-0.597510,1.733062,...,-0.756357,0.661187,-0.746191,-0.944316,-1.819972,0.408473,-0.438575,0.997185,0.393116,2.395806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5887,0.931222,1.060318,-0.034836,0.058898,-0.343167,0.208115,0.041661,0.394938,1.307151,1.495538,...,0.231426,2.363483,0.041093,0.947442,1.388563,-0.297332,-0.012953,0.434956,-1.196063,-0.638642
5888,0.033759,1.481159,-1.943646,-1.140197,-0.814531,-0.507689,0.869472,1.113435,-0.297249,1.052433,...,-1.780842,1.062261,-0.324040,0.893995,-1.403783,1.359306,0.017043,0.406932,-0.118585,1.993040
5889,-0.227356,-1.929745,0.262236,1.949443,0.411418,0.022323,-1.282553,0.062733,0.453313,-0.568709,...,2.915251,0.094499,-0.324420,-0.871167,1.132318,-1.259157,-0.301036,-0.777542,-0.906112,0.085534
5890,0.281614,0.820382,-2.588509,-0.481786,-0.318749,-0.155569,1.020637,0.820391,-0.300863,1.844694,...,-0.980848,0.975830,-1.311903,-0.458163,-0.167199,0.211024,0.184144,0.675549,0.254384,1.014646


In [245]:
pd.DataFrame(Z @ B)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,-1.474633,0.017306,1.338986,-0.900216,0.182907,-1.295644,2.783311,0.987771,-1.766683,-0.716554,...,0.146522,-0.400040,-0.324968,-0.169151,0.167464,-0.544671,-1.540237,0.147637,-0.933888,1.303343
1,-1.160526,1.218963,-0.143577,-0.414163,-0.362114,0.185558,0.243358,0.550979,-0.879077,2.250346,...,-0.134391,-0.833685,0.314062,0.499473,-1.185313,-0.980839,-1.583301,1.063001,0.665995,0.927597
2,-1.539776,1.227532,-2.109667,-0.682006,-0.183870,0.109060,1.520971,-0.056876,1.411115,0.256693,...,-1.993472,1.198721,-0.107615,-0.253636,-1.123311,0.014016,-0.392410,-0.427691,0.750461,2.183491
3,0.549465,-0.902659,0.779353,0.904666,0.472817,-0.058734,-1.066056,-1.604736,0.940198,-1.161394,...,-2.657157,-0.095349,0.716686,-0.477486,-0.424166,-0.656237,0.651159,-0.886212,-0.560591,0.923528
4,-0.024105,0.223273,-1.446224,-1.117613,0.149073,-0.040981,0.816818,0.453934,-0.597510,1.733062,...,-0.756357,0.661187,-0.746191,-0.944316,-1.819972,0.408473,-0.438575,0.997185,0.393116,2.395806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5887,0.931222,1.060318,-0.034836,0.058898,-0.343167,0.208115,0.041661,0.394938,1.307151,1.495538,...,0.231426,2.363483,0.041093,0.947442,1.388563,-0.297332,-0.012953,0.434956,-1.196063,-0.638642
5888,0.033759,1.481159,-1.943646,-1.140197,-0.814531,-0.507689,0.869472,1.113435,-0.297249,1.052433,...,-1.780842,1.062261,-0.324040,0.893995,-1.403783,1.359306,0.017043,0.406932,-0.118585,1.993040
5889,-0.227356,-1.929745,0.262236,1.949443,0.411418,0.022323,-1.282553,0.062733,0.453313,-0.568709,...,2.915251,0.094499,-0.324420,-0.871167,1.132318,-1.259157,-0.301036,-0.777542,-0.906112,0.085534
5890,0.281614,0.820382,-2.588509,-0.481786,-0.318749,-0.155569,1.020637,0.820391,-0.300863,1.844694,...,-0.980848,0.975830,-1.311903,-0.458163,-0.167199,0.211024,0.184144,0.675549,0.254384,1.014646


In [256]:
pd.DataFrame(svdres["v"])

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.177491,-0.030616,-0.117442,-0.290516,0.091422,-0.231867,-0.304756,0.151708,-0.045222,0.074408,...,-0.065073,0.090172,-0.090283,0.077909,0.056640,-0.009434,0.180497,0.049449,-0.165677,0.167650
1,0.093049,-0.184942,-0.158544,-0.009213,0.175392,0.091768,-0.202924,-0.103139,0.183736,-0.007996,...,0.382133,-0.047747,-0.135802,-0.493973,-0.085153,-0.056028,-0.356024,0.157118,-0.165677,0.167650
2,0.219910,0.123374,0.098651,-0.466178,-0.191182,-0.178881,0.510587,-0.410859,-0.171887,-0.082395,...,0.007576,-0.023934,0.027290,-0.025026,-0.003031,-0.008828,-0.072692,0.024175,-0.165677,0.167650
3,0.315039,0.281003,-0.268667,0.169794,-0.235057,-0.171600,-0.046893,0.054007,-0.074697,0.049759,...,-0.089474,-0.040158,0.064712,-0.052188,0.102649,-0.068409,0.088942,-0.071505,-0.165677,0.167650
4,0.045651,0.053069,-0.043128,0.199062,0.041016,0.026442,0.014069,-0.173721,0.259488,0.179553,...,0.038517,0.195320,-0.018134,0.159408,0.408517,0.361339,-0.090716,0.007183,-0.165677,0.167650
5,0.176812,-0.037460,0.023752,0.118690,0.100901,0.061268,-0.166273,-0.181984,0.091609,0.177258,...,-0.178735,0.077924,0.216092,0.122121,-0.495273,0.091949,0.297833,-0.118791,-0.165677,0.167650
6,-0.178758,-0.357693,-0.142183,0.064649,0.173731,0.118532,0.197576,-0.133709,-0.028311,-0.106049,...,-0.207855,0.129233,-0.203192,0.254934,0.003012,-0.068940,0.083106,0.003171,-0.165677,0.167650
7,-0.016757,-0.047299,-0.128663,0.179993,0.116109,0.004497,0.029049,-0.200074,-0.019563,-0.259006,...,0.099588,-0.035138,0.284058,-0.154448,0.017127,0.083266,0.320843,-0.052010,-0.165677,0.167650
8,0.123139,-0.254089,-0.042924,-0.203351,-0.133235,0.129448,0.125710,0.166770,0.354800,0.118921,...,-0.229437,-0.288986,0.106502,0.020896,0.202660,0.001181,-0.186949,-0.047172,-0.165677,0.167650
9,-0.046432,-0.126237,-0.113823,-0.042520,0.169603,-0.076227,0.376079,0.515754,-0.193513,0.279912,...,0.139760,0.030296,0.076010,-0.001690,-0.015110,-0.052359,-0.004798,0.011698,-0.165677,0.167650


In [259]:
pd.DataFrame(nsvdres["v"].T)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.177491,-0.030616,-0.117442,-0.290516,0.091422,-0.231867,0.304756,-0.151708,0.045222,-0.074408,...,0.065073,0.090172,0.090283,0.077909,0.056640,0.009434,-0.180497,0.049449,0.165677,0.167650
1,0.093049,-0.184942,-0.158544,-0.009213,0.175392,0.091768,0.202924,0.103139,-0.183736,0.007996,...,-0.382133,-0.047747,0.135802,-0.493973,-0.085153,0.056028,0.356024,0.157118,0.165677,0.167650
2,0.219910,0.123374,0.098651,-0.466178,-0.191182,-0.178881,-0.510587,0.410859,0.171887,0.082395,...,-0.007576,-0.023934,-0.027290,-0.025026,-0.003031,0.008828,0.072692,0.024175,0.165677,0.167650
3,0.315039,0.281003,-0.268667,0.169794,-0.235057,-0.171600,0.046893,-0.054007,0.074697,-0.049759,...,0.089474,-0.040158,-0.064712,-0.052188,0.102649,0.068409,-0.088942,-0.071505,0.165677,0.167650
4,0.045651,0.053069,-0.043128,0.199062,0.041016,0.026442,-0.014069,0.173721,-0.259488,-0.179553,...,-0.038517,0.195320,0.018134,0.159408,0.408517,-0.361339,0.090716,0.007183,0.165677,0.167650
5,0.176812,-0.037460,0.023752,0.118690,0.100901,0.061268,0.166273,0.181984,-0.091609,-0.177258,...,0.178735,0.077924,-0.216092,0.122121,-0.495273,-0.091949,-0.297833,-0.118791,0.165677,0.167650
6,-0.178758,-0.357693,-0.142183,0.064649,0.173731,0.118532,-0.197576,0.133709,0.028311,0.106049,...,0.207855,0.129233,0.203192,0.254934,0.003012,0.068940,-0.083106,0.003171,0.165677,0.167650
7,-0.016757,-0.047299,-0.128663,0.179993,0.116109,0.004497,-0.029049,0.200074,0.019563,0.259006,...,-0.099588,-0.035138,-0.284058,-0.154448,0.017127,-0.083266,-0.320843,-0.052010,0.165677,0.167650
8,0.123139,-0.254089,-0.042924,-0.203351,-0.133235,0.129448,-0.125710,-0.166770,-0.354800,-0.118921,...,0.229437,-0.288986,-0.106502,0.020896,0.202660,-0.001181,0.186949,-0.047172,0.165677,0.167650
9,-0.046432,-0.126237,-0.113823,-0.042520,0.169603,-0.076227,-0.376079,-0.515754,0.193513,-0.279912,...,-0.139760,0.030296,-0.076010,-0.001690,-0.015110,0.052359,0.004798,0.011698,0.165677,0.167650


In [299]:
iter_full_start = iter_full = 20

curfrac = 0
nposlast = np.inf
npos = -np.inf
if L3 is not None:
    L3_given = True
else:
    L3_given = False